In [1]:
import tabula
from tabula import read_pdf
from tabulate import tabulate
# from pypdf import PdfReader
# import pdfplumber
import tika
from tika import parser
# import pypdfium2 as pdfium
tika.initVM()
import sys
import json
import logging
import re
import zipfile
from pprint import pprint
import  usaddress 

import pandas as pd

from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import (SdkException, ServiceApiException, ServiceUsageException)
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions

C:\Users\pytho\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
from Utilities.pdf_utils  import *

In [3]:
def getData(file_path):

    zipfile = extract_pdf_and_zip(file_path)

    df_list , json_data = zip_processor(zipfile)
    # print(df_list)

    work_dict = json_getter(json_data)
#     pprint(work_dict)
    return df_list, work_dict

file_path = "C:\\noc\\SD%20Payor%20Scraping\\YourReport2.pdf"
dfs, work_d = getData(file_path)
# print(work_d)

Successfully extracted information from PDF. Printing H1 Headers:

['structuredData.json', 'tables/fileoutpart7.xlsx', 'tables/fileoutpart21.xlsx', 'tables/fileoutpart35.xlsx', 'tables/fileoutpart33.xlsx', 'tables/fileoutpart22.xlsx', 'tables/fileoutpart28.xlsx', 'tables/fileoutpart23.xlsx', 'tables/fileoutpart9.xlsx', 'tables/fileoutpart24.xlsx', 'tables/fileoutpart8.xlsx', 'tables/fileoutpart18.xlsx', 'tables/fileoutpart13.xlsx', 'tables/fileoutpart34.xlsx', 'tables/fileoutpart12.xlsx', 'tables/fileoutpart10.xlsx', 'tables/fileoutpart25.xlsx', 'tables/fileoutpart30.xlsx', 'tables/fileoutpart20.xlsx', 'tables/fileoutpart17.xlsx', 'tables/fileoutpart27.xlsx', 'tables/fileoutpart15.xlsx', 'tables/fileoutpart37.xlsx', 'tables/fileoutpart31.xlsx', 'tables/fileoutpart0.xlsx', 'tables/fileoutpart11.xlsx', 'tables/fileoutpart19.xlsx', 'tables/fileoutpart3.xlsx', 'tables/fileoutpart1.xlsx', 'tables/fileoutpart2.xlsx', 'tables/fileoutpart16.xlsx', 'tables/fileoutpart29.xlsx', 'tables/fileoutpa

In [4]:
tables = tabula.read_pdf(file_path,pages='all')

In [5]:
providers_df = tables[0]
print("PP", providers_df.iloc[0,0])
print(providers_df.shape[0])
providers_list = []
for i in range(providers_df.shape[0]):
    provider_dict = {}
    provider_dict['Name'] = providers_df.iloc[i,0]
    provider_dict['Pin'] = providers_df.iloc[i,1]
    provider_dict['Amount'] = providers_df.iloc[i,3]
    providers_list.append(provider_dict)
print(len(providers_list))
providers_list
    

PP ADERONKE O OGUNBAMERU DDS
3
3


[{'Name': 'ADERONKE O OGUNBAMERU DDS', 'Pin': 4726606, 'Amount': '$1,751.20'},
 {'Name': 'NICHOLAS T BUSHEY DDS', 'Pin': 4780373, 'Amount': '$559.00'},
 {'Name': 'ARTHUR E WILLIAMS II DDS', 'Pin': 5499471, 'Amount': '$369.90'}]

In [6]:
texts = '\n'.join([t['Text'] for t in work_d['elements']])
texts

'Page: \n1 of 11 \nCHILDREN\'S DENTAL AT PRESTON TRAIL, \nPC \nTIN: \nXXXXXXX4771 \nTrace Number: \n823100000160773 \nTrace Amount: \n$2,932.10 \nCHILDREN\'S DENTAL AT PRESTON TRAIL, PC \n3602 MATLOCK RD STE 208 \nARLINGTON TX 76015-3600 \nNotes: Enclosed is a group payment to the XXXXXXX4771. Below is a itemization of the check. Please refer to the attached statements of details. \nBreakdown of Payment(s) Issued: \nNAME \nPIN \nISSUED AMT \nADERONKE O OGUNBAMERU DDS \n0004726606 \n$1,751.20 \nNICHOLAS T BUSHEY DDS \n0004780373 \n$559.00 \nARTHUR E WILLIAMS II DDS \n0005499471 \n$369.90 \nAetna Life Insurance Company or an Affiliated Company as Agent for Specified Payer(s) \nP.O. BOX 981106 \nEL PASO TX 79998-1106 \nUSA \nID No: XXXXXXX4771 \nSeq No: 000000004 \nTrace No: 000160773 \nAcct: 09046 \n51 -44 \n119 CT04-10-2023 \nPAYER MULTIPLENON-NEGOTIABLE NON-NEGOTIABLE PAY Two Thousand Nine Hundred Thirty Two Dollars and 10/100 \nVOID AFTER ONE YEAR \n******$2,932.10 \nTO THE ORDER OF \

In [7]:
tt = texts.split('Breakdown of Payment(s) Issued:')[1].split('TOTAL ISSUED AMOUNT')[0]
print(tt)

 
NAME 
PIN 
ISSUED AMT 
ADERONKE O OGUNBAMERU DDS 
0004726606 
$1,751.20 
NICHOLAS T BUSHEY DDS 
0004780373 
$559.00 
ARTHUR E WILLIAMS II DDS 
0005499471 
$369.90 
Aetna Life Insurance Company or an Affiliated Company as Agent for Specified Payer(s) 
P.O. BOX 981106 
EL PASO TX 79998-1106 
USA 
ID No: XXXXXXX4771 
Seq No: 000000004 
Trace No: 000160773 
Acct: 09046 
51 -44 
119 CT04-10-2023 
PAYER MULTIPLENON-NEGOTIABLE NON-NEGOTIABLE PAY Two Thousand Nine Hundred Thirty Two Dollars and 10/100 
VOID AFTER ONE YEAR 
******$2,932.10 
TO THE ORDER OF 
Bank of America 
CHILDREN'S DENTAL AT PRESTON TRAIL, PC 3602 MATLOCK RD STE 208 ARLINGTON TX 76015-3600 
766 (10-02) 
Payment was made via Electronic Funds Transfer 
Payment Address: 
CHILDREN'S DENTAL AT PRESTON TRAIL, PC 
3602 MATLOCK RD STE 208 
ARLINGTON TX 76015-3600 
Summary of Claim Payment 
Printed: 
04/10/2023 
Page: 
2 of 11 
CHILDREN'S DENTAL AT PRESTON TRAIL, 
PC 
TIN: 
XXXXXXX4771 
Trace Number: 
823100000160773 
Trace Amount:

In [8]:
# texts = []
# for t in work_d['elements']:
#     texts.append(t['Text'])

text_ele = [item.get('Text') for item in work_d['elements']]

    
for text in text_ele:
    if "Patient Name:" in text:
        print(text)


Patient Name: MICHAEL-GI A ALOMBAH (son) 
Patient Name: IMANI M ROBY (daughter) 
Patient Name: IMANI M ROBY (daughter) 
Patient Name: QAMAR J ROBY (daughter) 
Patient Name: QAMAR J ROBY (daughter) 
Patient Name: JAKE R HURGUY (son) 
Patient Name: HASHIM KHALED (son) 
Patient Name: WISSAM KHALED (son) 
Patient Name: WISSAM KHALED (son) 
Patient Name: KHLOE E STACK (daughter) 
Patient Name: GIOVANNI L MENDEZ (son) 
Patient Name: GIOVANNI L MENDEZ (son) 
Patient Name: NICHOLAS D TOUSSAINT (son) 
Patient Name: BRENNAN C THORNHILL (son) 


In [9]:
def get_basic_details(filepath):
    tabula_dfs = tabula.read_pdf(filepath,guess=False,pages=1,stream=True,encoding="utf-8",
        area=[(18, 26, 81, 233), (33,363,172,543), (128,26,193,265)],multiple_tables=True)
    
    add_col = tabula_dfs[0].columns[0]
    payer_add= f"aetna {add_col} {' '.join(tabula_dfs[0][add_col ].to_list())}"
    
    payer_address_details = extract_address_details(payer_add)

    payment_col = tabula_dfs[2].columns[0]
    payment_add = f"{payment_col} {' '.join(tabula_dfs[2][payment_col ].to_list())}"
    payee_address_details = extract_address_details(payment_add)

    other_det_col = tabula_dfs[1].columns[0]
    other_det_list = tabula_dfs[1][other_det_col ].to_list()
    for item in other_det_list:
        if 'Trace Number:' in item:
            trace_num = item.split('Trace Number:')[1].strip()

        if 'Trace Amount:' in item:
            trace_amount = item.split('Trace Amount:')[1].strip()
            
    master_dict = {
        "Payer": payer_address_details.get("Item", ""),
        "PayerName": 'aetna',
        "PayerAddress": payer_address_details.get("AddressElements", ""),
        "PayerCity": payer_address_details.get("PlaceName", ""),
        "PayerState": payer_address_details.get("StateName", ""),
        "PayerZip": payer_address_details.get("ZipCode", ""),
        "Payee": payee_address_details.get("Item", ""),
        "PayeeName": payee_address_details.get("ItemName", ""),
        "PayeeAddress": payee_address_details.get("AddressElements", ""),
        "PayeeCity": payee_address_details.get("PlaceName", ""),
        "PayeeState": payee_address_details.get("StateName", ""),
        "PayeeZip": payee_address_details.get("ZipCode", ""),
        "EFT_CheckNumber": trace_num
    }
    
    return master_dict
    
my_dict = get_basic_details(file_path)
my_dict

{'Payer': 'aetna P.O. BOX 981106 EL PASO TX 79998-1106 USA',
 'PayerName': 'aetna',
 'PayerAddress': 'aetna P.O. BOX 981106',
 'PayerCity': 'EL PASO',
 'PayerState': 'TX',
 'PayerZip': '79998-1106',
 'Payee': "CHILDREN'S DENTAL AT PRESTON TRAIL, PC 3602 MATLOCK RD STE 208 ARLINGTON TX 76015-3600",
 'PayeeName': "CHILDREN'S DENTAL AT PRESTON TRAIL, PC",
 'PayeeAddress': '3602 MATLOCK RD STE 208',
 'PayeeCity': 'ARLINGTON',
 'PayeeState': 'TX',
 'PayeeZip': '76015-3600',
 'EFT_CheckNumber': '823100000160773'}

In [10]:
provider_ele = texts.split('Total Payment to:')

In [11]:
def extract_notes(text:str, provider_name:str)-> str:
    remark = ''
    if 'Remarks' in text:
        remark = text.split('\nRemarks:')[1]
        if 'For questions regarding this claim or if you wish a review of this decision:' in remark:
            remark = remark.split('For questions regarding this claim or if you wish a review of this decision:')[0]
        if provider_name in remark:
            remark = remark.split(provider_name)[0]
    return remark

In [12]:
all_patients = []
for x in range(len(provider_ele[:-1])):
    PATIENT=provider_ele[x].split('Patient Name:')
    
    for j in range(len(PATIENT)):

                


#         if 'ISSUED AMT:' not in PATIENT[j]:
#             continue
        for i in PATIENT[j].split('Claim ID:')[1:]:
            
            patient_dict = {}
            patient_name = PATIENT[j].split('Claim ID')[0].split('\n')[0].strip()
            patient_name, relation = patient_name.split('(')
            patient_dict['PatientName'] = patient_name.strip()
            patient_dict['Relationship'] = relation.replace(')', '').strip()
            patient_dict['ClaimId'] = i.split(' ')[1].strip()
            patient_dict['Recd'] = i.split('Recd:')[1].split(' ')[1].strip()
            patient_dict['MemberID'] = i.split('Member ID:')[1].split(' ')[1].strip()
            patient_dict['PatientAccount'] = i.split('Patient Account:')[1].split(' ')[1].strip()
            patient_dict['Member'] = i.split('Member:')[1].split('\n')[0].strip()
            patient_dict['GroupName'] = i.split('Group Name:')[1].split('Group Number')[0].split('\n')[0].strip()
            patient_dict['GroupNumber'] = i.split('Group Number:')[1].split('Product')[0].split('\n')[0].strip()
            patient_dict['Product'] = i.split('Product:')[1].split('Network ID:')[0].split('\n')[0].strip()
            patient_dict['NetworkID'] = i.split('Network ID:')[1].strip().split(' ')[0].strip()
            provider_name = provider_ele[x+1].split('\n')[0].split('$')[0].strip()
            patient_dict['Provider'] = provider_name.strip()
            patient_dict['TransactionFee'] = '$0.00'
            
            
            patient_dict['notes'] = extract_notes(i, provider_name)
            patient_dict = {**patient_dict, **my_dict} 
            all_patients.append(patient_dict)


In [13]:
all_patients.__len__()

10

In [14]:
all_patients

[{'PatientName': 'MICHAEL-GI A ALOMBAH',
  'Relationship': 'son',
  'ClaimId': 'EHFC4680W00',
  'Recd': '04/03/23',
  'MemberID': 'W461594992',
  'PatientAccount': '443443935',
  'Member': 'ATANGA C ALOMBAH',
  'GroupName': 'CVS PHARMACY, INC.',
  'GroupNumber': '0141975-10-001 A ZHYF"0',
  'Product': 'Aetna Dental® PPO',
  'NetworkID': '00000',
  'Provider': 'ADERONKE O OGUNBAMERU DDS',
  'TransactionFee': '$0.00',
  'notes': " \n1 - This amount is over the recognized charge for this service. We determine the recognized charge based on the geographic area, the member's plan and we calculate it based on either: \n-The FAIR Health percentile \n-The plan's nonparticipating fee schedule \nWe believe our payment to you was fair. If you have more information or questions, let us know. Use the number on this statement. [551] \n2 - The member's dental plan of benefits limits how often fluoride treatments can be covered in a specific period of time. This fluoride treatment exceeds this limit. 

In [15]:
providers_list

[{'Name': 'ADERONKE O OGUNBAMERU DDS', 'Pin': 4726606, 'Amount': '$1,751.20'},
 {'Name': 'NICHOLAS T BUSHEY DDS', 'Pin': 4780373, 'Amount': '$559.00'},
 {'Name': 'ARTHUR E WILLIAMS II DDS', 'Pin': 5499471, 'Amount': '$369.90'}]

In [16]:
eobpatient_list = []
for e in all_patients:
    eobpatient_dict = {}
    pfname, plname = e['PatientName'].split(' ')[0], e['PatientName'].split(' ')[-1]
    eobpatient_dict['ProviderClaimId']=  ''
    eobpatient_dict['PatientFullName']=  e['PatientName']
    eobpatient_dict['PayerClaimId'] =  e['ClaimId']
    
    eobpatient_dict['MemberNo'] = e['MemberID']
#     eobpatient_dict['PayerPaid'] = ''
    eobpatient_dict['SubscriberID'] = ''
    eobpatient_dict['RenderingProviderFirstName'] = e['Provider'].split(' ',1)[0].strip()
    eobpatient_dict['RenderingProviderLastName'] = e['Provider'].split(' ',1)[1].strip()
    eobpatient_dict['PatientName'] =  e['PatientName']
    eobpatient_dict['ServiceDate'] =  ''
    eobpatient_dict['EFT_CheckNumber'] = e['EFT_CheckNumber']
    eobpatient_dict['PatientFirstName'] =  pfname
    eobpatient_dict['PatientLastName'] =  plname
    eobpatient_dict['Enrollee_ClaimID'] =  ''
    eobpatient_dict['OtherAdjustments'] =  ''
    eobpatient_dict['PlanType'] = ''
#     eobpatient_dict['RenderingProviderId'] = ''
    for p in providers_list[:-1]:
        total_amount = providers_list[-1]['Amount']
        if p['Name'] in str(e['Provider']):
            eobpatient_dict['RenderingProviderID'] = p['Pin']
            eobpatient_dict['PayerPaid'] = total_amount
            
    
    eobpatient_list.append(eobpatient_dict)

In [17]:
eobpatient_list

[{'ProviderClaimId': '',
  'PatientFullName': 'MICHAEL-GI A ALOMBAH',
  'PayerClaimId': 'EHFC4680W00',
  'MemberNo': 'W461594992',
  'SubscriberID': '',
  'RenderingProviderFirstName': 'ADERONKE',
  'RenderingProviderLastName': 'O OGUNBAMERU DDS',
  'PatientName': 'MICHAEL-GI A ALOMBAH',
  'ServiceDate': '',
  'EFT_CheckNumber': '823100000160773',
  'PatientFirstName': 'MICHAEL-GI',
  'PatientLastName': 'ALOMBAH',
  'Enrollee_ClaimID': '',
  'OtherAdjustments': '',
  'PlanType': '',
  'RenderingProviderID': 4726606,
  'PayerPaid': '$369.90'},
 {'ProviderClaimId': '',
  'PatientFullName': 'IMANI M ROBY',
  'PayerClaimId': 'EHPC45YT800',
  'MemberNo': 'W145895339',
  'SubscriberID': '',
  'RenderingProviderFirstName': 'ADERONKE',
  'RenderingProviderLastName': 'O OGUNBAMERU DDS',
  'PatientName': 'IMANI M ROBY',
  'ServiceDate': '',
  'EFT_CheckNumber': '823100000160773',
  'PatientFirstName': 'IMANI',
  'PatientLastName': 'ROBY',
  'Enrollee_ClaimID': '',
  'OtherAdjustments': '',
  'Pl

In [18]:
for x in range(len(provider_ele[:-1])):
    PATIENT=provider_ele[x].split(')\n')
    for p in all_patients:
        noun = p.get('PatientName')
        for j in range(len(PATIENT)):
            notes = ''
            p_name = PATIENT[j].split('(')[0]
            print(p_name)
            if noun == p_name:
                print(noun)
                if p_name in  PATIENT[j-1].split('(') and 'Claim ID:' not in PATIENT[j]:
                #sudo work on prev patient
                    if 'Remarks:' in  PATIENT[j]:
                        notes = extract_notes(PATIENT[j])
                        print(notes)
                        p['notes'] = notes


            

Page: 
1 of 11 
CHILDREN'S DENTAL AT PRESTON TRAIL, 
PC 
TIN: 
XXXXXXX4771 
Trace Number: 
823100000160773 
Trace Amount: 
$2,932.10 
CHILDREN'S DENTAL AT PRESTON TRAIL, PC 
3602 MATLOCK RD STE 208 
ARLINGTON TX 76015-3600 
Notes: Enclosed is a group payment to the XXXXXXX4771. Below is a itemization of the check. Please refer to the attached statements of details. 
Breakdown of Payment
Page: 
1 of 11 
CHILDREN'S DENTAL AT PRESTON TRAIL, 
PC 
TIN: 
XXXXXXX4771 
Trace Number: 
823100000160773 
Trace Amount: 
$2,932.10 
CHILDREN'S DENTAL AT PRESTON TRAIL, PC 
3602 MATLOCK RD STE 208 
ARLINGTON TX 76015-3600 
Notes: Enclosed is a group payment to the XXXXXXX4771. Below is a itemization of the check. Please refer to the attached statements of details. 
Breakdown of Payment
Page: 
1 of 11 
CHILDREN'S DENTAL AT PRESTON TRAIL, 
PC 
TIN: 
XXXXXXX4771 
Trace Number: 
823100000160773 
Trace Amount: 
$2,932.10 
CHILDREN'S DENTAL AT PRESTON TRAIL, PC 
3602 MATLOCK RD STE 208 
ARLINGTON TX 76015-36

In [19]:
all_patients

[{'PatientName': 'MICHAEL-GI A ALOMBAH',
  'Relationship': 'son',
  'ClaimId': 'EHFC4680W00',
  'Recd': '04/03/23',
  'MemberID': 'W461594992',
  'PatientAccount': '443443935',
  'Member': 'ATANGA C ALOMBAH',
  'GroupName': 'CVS PHARMACY, INC.',
  'GroupNumber': '0141975-10-001 A ZHYF"0',
  'Product': 'Aetna Dental® PPO',
  'NetworkID': '00000',
  'Provider': 'ADERONKE O OGUNBAMERU DDS',
  'TransactionFee': '$0.00',
  'notes': " \n1 - This amount is over the recognized charge for this service. We determine the recognized charge based on the geographic area, the member's plan and we calculate it based on either: \n-The FAIR Health percentile \n-The plan's nonparticipating fee schedule \nWe believe our payment to you was fair. If you have more information or questions, let us know. Use the number on this statement. [551] \n2 - The member's dental plan of benefits limits how often fluoride treatments can be covered in a specific period of time. This fluoride treatment exceeds this limit. 

In [20]:
add_col = tabula_dfs[0].columns[0]
payer_add= f"aetna {add_col} {' '.join(tabula_dfs[0][add_col ].to_list())}"
payer_add

payment_col = tabula_dfs[2].columns[0]
payment_add = f"{payment_col} {' '.join(tabula_dfs[2][payment_col ].to_list())}"
payment_add

other_det_col = tabula_dfs[1].columns[0]
other_det_list = tabula_dfs[1][other_det_col ].to_list()
filtered_list = [item for item in other_det_list if 'Trace Number:' in item and 'Trace Amount:' in item]
filtered_list

NameError: name 'tabula_dfs' is not defined

In [ ]:
tabula_dfs[2]

In [ ]:
payment_col = tabula_dfs[2].columns[0]
payment_add = f"{payment_col} {' '.join(tabula_dfs[2][payment_col ].to_list())}"
payment_add

In [ ]:
payment_col

In [ ]:
tabula_dfs[2][payment_col ].to_list()

In [ ]:
other_det_col = tabula_dfs[1].columns[0]
other_det_list = tabula_dfs[1][other_det_col ].to_list()
filtered_list = [item for item in other_det_list if 'Trace Number:' in item and 'Trace Amount:' in item]
filtered_list

In [ ]:
tabula_dfs[1]

In [ ]:
other_det_list

In [ ]:
trace_amount